# Spammer: Harvesting emails and phone numbers
Extract phone numbers and email addresses from web pages (HTML).

In [13]:
from __future__ import print_function

import os
import pandas as pd
import re


## Data
Data is split into training and testing sets. Data consists of HTML files, with embedded e-mail addresses and phone numbers.

In [10]:
TRAIN_FOLDER = 'dev'
TEST_FOLDER = 'test'
TRAIN_LABELS_FILE = 'devGOLD'

In [25]:
train_files = os.listdir(TRAIN_FOLDER)
print(sorted(train_files))

['aiken', 'ashishg', 'balaji', 'bgirod', 'cheriton', 'christos', 'dabo', 'dlwh', 'dm', 'engler', 'eroberts', 'fedkiw', 'hager', 'hanrahan', 'horowitz', 'jks', 'jurafsky', 'jure', 'knuth', 'koller', 'kosecka', 'kunle', 'lam', 'latombe', 'levoy', 'manning', 'nass', 'nick', 'nickm']


These are the HTML files in our training set. Let's load all the raw text.

In [27]:
# Dictionary comprehension FTW
train_data = {fn:open(os.path.join(TRAIN_FOLDER, fn), 'r').read() for fn in train_files}

Ok, great. Now for each training file we have the raw HTML. Let's take a look at the training labels next.

In [21]:
with open(TRAIN_LABELS_FILE, 'r') as f:
    for line in f.readlines()[:5]:
        print(line)

ashishg	e	ashishg@stanford.edu

ashishg	e	rozm@stanford.edu

ashishg	p	650-723-1614

ashishg	p	650-723-4173

ashishg	p	650-814-1478



We can see that our labels come in a white-space delimited file. From context we can tell that the first column is the training file, the second column is the type ('e' for email, 'p' for phone-number), and the third column is the extracted value. This is a good format for a pandas data frame, so let's load the label data using pandas.

In [24]:
train_labels = pd.read_csv(TRAIN_LABELS_FILE, delim_whitespace=True,
                           header=None, names=['file_name','type','value'])
print(train_labels.head())

  file_name type                 value
0   ashishg    e  ashishg@stanford.edu
1   ashishg    e     rozm@stanford.edu
2   ashishg    p          650-723-1614
3   ashishg    p          650-723-4173
4   ashishg    p          650-814-1478


Great, looking good!

## Train our model
Let's start developing our algorithm. We will be using cascades of regular expressions. So instead of having a machine learning model automatically learn from the training data through some sort of optimization, I will manually update the model to fit the training data, and hopefully generalize to the test data.

## Test our model
Let's see how well the algorithm we've developed works on the test data.